In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import scale
from sklearn.utils import resample
from hyperopt import STATUS_OK, hp, fmin, tpe, Trials, space_eval

from time import time
import operator

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import itertools
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.ERROR)

### Start training

In [2]:
# Import data
def load_data():
    full_data = pd.read_csv("X.csv")
    train_y = pd.read_csv("ytr.csv")
    # Rename columns to something more interpretable
    columns = (["reflectance_" + str(i) for i in range(7)]
               + ["solar_" + str(i) for i in range(5)] + ["id"])
    full_data.columns = columns
    # Add y to the data frame
    split = 98000
    y_id_dict = train_y.set_index("Id")["y"].to_dict()
    full_data.loc[:(split-1), "y"] = full_data.loc[:(split-1), "id"].map(y_id_dict)

    train, test = full_data[:split], full_data[split:]
    return (train, test)

#columns = (["id"] + ["reflectance_" + str(i) for i in range(7)]
#           + ["solar_" + str(i) for i in range(5)] + ["y"])
#full_data = pd.read_csv("MODIS.csv", header=None, names=columns)
#split = 98000
#train, test = full_data[:split].copy(), full_data[split:].copy()
#train_copy, test_copy = load_data()
train, test = load_data()

# Parameters
n_threads = -1
random_seed = 22

In [3]:
train.head()

,reflectance_0,reflectance_1,reflectance_2,reflectance_3,reflectance_4,reflectance_5,reflectance_6,solar_0,solar_1,solar_2,solar_3,solar_4,id,y
0,1.580642,2.482233,5.887092,4.732722,4.408482,3.830171,4.388508,22.572888,63.58724,88.05048,4.495216,-50.699904,1,-3.998082
1,2.338455,3.627796,4.723716,3.324726,2.743442,4.727652,2.810193,22.572888,63.58724,88.05048,4.495216,-50.699904,1,-3.998082
2,2.224569,3.522241,6.188831,4.389783,4.177616,4.945918,4.122848,22.572888,63.58724,88.05048,4.495216,-50.699904,1,-3.998082
3,1.717218,2.712012,5.024211,3.944907,3.393424,3.931973,3.489578,22.572888,63.58724,88.05048,4.495216,-50.699904,1,-3.998082
4,2.378857,3.644976,4.515292,3.223825,2.739952,4.599662,2.781574,22.572888,63.58724,88.05048,4.495216,-50.699904,1,-3.998082


In [4]:
#train, test = train_copy[19600:].copy(), train_copy[:19600].copy()

### Preprocessing

In [5]:
cols_excl = ["id", "y"]
cols_orig = [c for c in train.columns if c not in cols_excl]

# Standardise data for LR
train[cols_orig] = scale(train[cols_orig])
test[cols_orig] = scale(test[cols_orig])

### Cross-validation

In [6]:
cols_dnn = cols_orig

models_weights = {"dnn_1": 1.0}
models_cols = {"dnn_1": cols_dnn}

# Scoring function in the hyperopt hyperparameters tuning.
def scoring_function(parameters):
    print("Training the model with parameters: ")
    print(parameters)
    average_RMSE = 0.0
    n_splits = 5
    
    # Generate random integer for model_dir
    random_int = np.random.randint(1000)
    
    kf = KFold(n_splits=n_splits)
    nb_fold = 0
    for train_index, validation_index in kf.split(train):
        nb_fold += 1
        train_fold, validation_fold = train.loc[train_index], train.loc[validation_index] 

        feature_cols = [tf.contrib.layers.real_valued_column(k) for k in cols_dnn]
        
        model_dir = ("./log_"
                     + str(parameters["steps"]) + "_"
                     + str(parameters["nb_neurons_1"]) + "_"
                     #+ str(parameters["nb_neurons_2"])
                     + str(nb_fold) + "_"
                     + str(random_int)
                    )
        
        # Tune number of layers
        model_dnn = tf.contrib.learn.DNNRegressor(feature_columns=feature_cols,
                                                  hidden_units=[10],
                                                                #parameters["nb_neurons_2"]],
                                                  #optimizer=tf.train.ProximalAdagradOptimizer(
                                                   #   learning_rate=0.1,
                                                    #  l1_regularization_strength=0.001),
                                                  model_dir=model_dir)

        def input_fn(data_set):
            feature_cols = {k: tf.constant(data_set[k].values) for k in cols_dnn}
            labels = tf.constant(data_set["y"].values)
            return feature_cols, labels
        
        model_dnn.fit(input_fn=lambda: input_fn(train_fold), steps=900)

        train_pred = train_fold[["id"]].assign(y_hat=0)
        #for i, m in models.items():
        temp = model_dnn.predict(input_fn=lambda: input_fn(train_fold))
        # .predict() returns an iterator; convert to an array
        y_hat = np.array(list(itertools.islice(temp, 0, None)))
        train_pred["y_hat"] = y_hat

        # Use median value by id
        y_hat_med = train_pred.groupby("id").median()["y_hat"].to_dict()

        RMSE = np.sqrt(mean_squared_error(train_pred["id"].map(y_hat_med).values, train_fold["y"]))
        
        # Prune outliers
        RMSE_decreasing = True
        count = 0
        while (RMSE_decreasing):
            count +=1
            train_pred["y_med"] = train_pred["id"].map(y_hat_med)

            # Distance from the median for each bag
            train_pred["score"] = (train_pred["y_hat"] - train_pred["y_med"])**2
            # Rank of each instance by bag
            train_pred["rank"] = train_pred.groupby("id")["score"].rank()
            bag_size_dict = train_pred.groupby("id")["score"].count().to_dict()
            train_pred["bag_size"] = train_pred["id"].map(bag_size_dict)
            train_pred["rank"] = train_pred["rank"] / train_pred["bag_size"]

            # Remove outliers
            outliers_index = train_pred["rank"] > (1 - parameters["outliers_threshold"])
            train_fold = train_fold.loc[~outliers_index, :].reset_index(drop=True)

            model_dnn = tf.contrib.learn.DNNRegressor(feature_columns=feature_cols,
                                                      hidden_units=[10],
                                                                    #parameters["nb_neurons_2"]],
                                                      #optimizer=tf.train.ProximalAdagradOptimizer(
                                                       #   learning_rate=0.1,
                                                        #  l1_regularization_strength=0.001),
                                                      model_dir=model_dir)

            model_dnn.fit(input_fn=lambda: input_fn(train_fold), steps=900)

            # Compute new RMSE
            train_pred = train_fold[["id"]].assign(y_hat=0)
            
            #for i, m in models.items():
            temp = model_dnn.predict(input_fn=lambda: input_fn(train_fold))
            # .predict() returns an iterator; convert to an array
            y_hat = np.array(list(itertools.islice(temp, 0, None)))
            train_pred["y_hat"] = y_hat

            # Use median value by id
            y_hat_med = train_pred.groupby("id").median()["y_hat"].to_dict()

            new_RMSE = np.sqrt(mean_squared_error(train_pred["id"].map(y_hat_med), train_fold["y"]))
            print("Pruning {0} RMSE: {1}".format(count, new_RMSE))

            if (abs(new_RMSE - RMSE) > parameters["gain_threshold"]):
                RMSE = new_RMSE
            else:
                RMSE_decreasing = False
        
        # Second bigger DNN
        model_dnn_1 = tf.contrib.learn.DNNRegressor(feature_columns=feature_cols,
                                                    hidden_units=[parameters["nb_neurons_1"]])
                                                    #model_dir=model_dir)
        model_dnn_1.fit(input_fn=lambda: input_fn(train_fold), steps=parameters["steps"])
        
        # Changed to model_dnn instead of model_dnn_1
        models = {"dnn_1": model_dnn_1}
        
        # Compute RMSE on validation set
        validation_pred = validation_fold[["id"]].assign(y_hat=0).reset_index(drop=True)
        for i, m in models.items():
            temp = m.predict(input_fn=lambda: input_fn(validation_fold))
            # .predict() returns an iterator; convert to an array
            y_hat = np.array(list(itertools.islice(temp, 0, None)))
            validation_pred["y_hat"] += models_weights[i] * y_hat
            
        # Use median value by id
        y_hat_med = validation_pred.groupby("id").median()["y_hat"].to_dict()
        
        RMSE = np.sqrt(mean_squared_error(validation_pred["id"].map(y_hat_med).values, validation_fold["y"]))
        average_RMSE += RMSE
        print("Validation fold {0} RMSE: {1}".format(nb_fold, RMSE))

    average_RMSE /= n_splits

    print("Cross-validation score: {0}\n".format(average_RMSE))
    
    return {"loss": average_RMSE, "status": STATUS_OK}

In [7]:
t0 = time()

# Grid to pick parameters from.
parameters_grid = {"steps"             : hp.choice("steps", np.arange(10000, 10100, 100, dtype=int)),
                   "nb_neurons_1"      : hp.choice("nb_neurons_1", np.arange(24, 25, 1, dtype=int)),
                   "outliers_threshold": hp.quniform("outliers_threshold", 0.05, 0.051, 0.01),
                   "gain_threshold"    : hp.quniform("gain_threshold", 0.005, 0.01, 0.005)
                   #"nb_neurons_2": hp.choice("nb_neurons_2", np.arange(5, 10, 1, dtype=int))
                  }
# Record the information about the cross-validation.
trials = Trials()

best = fmin(scoring_function, parameters_grid, algo=tpe.suggest, max_evals=1, 
            trials=trials)

computing_time = time() - t0

Training the model with parameters: 
{'gain_threshold': 0.01, 'steps': 10000, 'nb_neurons_1': 24, 'outliers_threshold': 0.05}
Pruning 1 RMSE: 0.699313533454
Pruning 2 RMSE: 0.688515929788
Pruning 3 RMSE: 0.68023760989


KeyboardInterrupt: 

In [ ]:
Pruning 1 RMSE: 0.693409560231
Pruning 2 RMSE: 0.677430813304
Pruning 3 RMSE: 0.668280328097
Pruning 4 RMSE: 0.662140161136
Pruning 5 RMSE: 0.656442073871
Pruning 6 RMSE: 0.651894112733
Pruning 7 RMSE: 0.647910685952
Pruning 8 RMSE: 0.644673528421
Pruning 9 RMSE: 0.642254075294
Validation fold 1 RMSE: 0.646948737672
Pruning 1 RMSE: 0.711768146502
Pruning 2 RMSE: 0.689930723516

In [ ]:
# 1 DNN with model_dir corrected
Training the model with parameters: 
{'gain_threshold': 0.005, 'steps': 1000, 'nb_neurons_1': 11, 'outliers_threshold': 0.05}
Pruning 1 RMSE: 0.708254362329
Pruning 2 RMSE: 0.690809364036
Pruning 3 RMSE: 0.681259286244
Pruning 4 RMSE: 0.675679762642
Pruning 5 RMSE: 0.670965511179
Validation fold 1 RMSE: 0.64861798817
Pruning 1 RMSE: 0.658292571898
Pruning 2 RMSE: 0.645737414652
Pruning 3 RMSE: 0.63797501753
Pruning 4 RMSE: 0.632583378549
Pruning 5 RMSE: 0.627584029004
Validation fold 2 RMSE: 0.723643911448
Pruning 1 RMSE: 0.687961545764
Pruning 2 RMSE: 0.672401033846
Pruning 3 RMSE: 0.660611288466
Pruning 4 RMSE: 0.650237174155
Pruning 5 RMSE: 0.64198570573
Pruning 6 RMSE: 0.636986731147
Validation fold 3 RMSE: 0.719981056486
Pruning 1 RMSE: 0.697781938911
Pruning 2 RMSE: 0.681268268398
Pruning 3 RMSE: 0.671483140797
Pruning 4 RMSE: 0.666088232133
Pruning 5 RMSE: 0.661858777702
Validation fold 4 RMSE: 0.715854578144
Pruning 1 RMSE: 0.66224449426
Pruning 2 RMSE: 0.644538081642
Pruning 3 RMSE: 0.634079022739
Pruning 4 RMSE: 0.624630852786
Pruning 5 RMSE: 0.619603795183
Pruning 6 RMSE: 0.61598970139
Validation fold 5 RMSE: 0.762927716346
Cross-validation score: 0.714205050119

Training the model with parameters: 
{'gain_threshold': 0.01, 'steps': 1100, 'nb_neurons_1': 10, 'outliers_threshold': 0.05}
Pruning 1 RMSE: 0.713960264442
Pruning 2 RMSE: 0.701637487701
Pruning 3 RMSE: 0.69375349128
Validation fold 1 RMSE: 0.641879797964
Pruning 1 RMSE: 0.671532366166
Pruning 2 RMSE: 0.656442799709
Pruning 3 RMSE: 0.645330625403
Pruning 4 RMSE: 0.638014896705
Validation fold 2 RMSE: 0.741045867638
Pruning 1 RMSE: 0.690110283558
Pruning 2 RMSE: 0.678068607072
Pruning 3 RMSE: 0.662923307388
Pruning 4 RMSE: 0.654881538105
Validation fold 3 RMSE: 0.687377107282
Pruning 1 RMSE: 0.70616839345
Pruning 2 RMSE: 0.692895897509
Pruning 3 RMSE: 0.686021953655
Validation fold 4 RMSE: 0.688327537634
Pruning 1 RMSE: 0.66943420746
Pruning 2 RMSE: 0.65286815522
Pruning 3 RMSE: 0.638691155949
Pruning 4 RMSE: 0.627600556022
Pruning 5 RMSE: 0.622316478
Validation fold 5 RMSE: 0.765910014041
Cross-validation score: 0.704908064912

In [8]:
min(trials.losses())

In [ ]:
# Save the best parameters as a csv.
best_parameters = pd.DataFrame({key: [value] for (key, value) in 
                                zip(space_eval(parameters_grid, best).keys(),
                                    space_eval(parameters_grid, best).values())})
# Add the corresponding score.
best_parameters["score"] = min(trials.losses())
best_parameters.to_csv("best_parameters_6.csv", encoding="utf-8", index=False)

best_parameters

### Training models

In [8]:
cols_dnn = cols_orig

models_weights = {"dnn_1": 1.0}
models_cols = {"dnn_1": cols_dnn}

best_parameters = pd.read_csv("best_parameters_6.csv", encoding="utf-8")

feature_cols = [tf.contrib.layers.real_valued_column(k) for k in cols_dnn]

model_dir = "./log_submit_6"
        
# Tune number of layers
model_dnn = tf.contrib.learn.DNNRegressor(feature_columns=feature_cols,
                                          hidden_units=[best_parameters["nb_neurons_1"][0]],
                                          model_dir=model_dir)

def input_fn(data_set):
    feature_cols = {k: tf.constant(data_set[k].values) for k in cols_dnn}
    labels = tf.constant(data_set["y"].values)
    return feature_cols, labels

model_dnn.fit(input_fn=lambda: input_fn(train), steps=best_parameters["steps"][0])
        

train_pred = train[["id"]].assign(y_hat=0)
temp = model_dnn.predict(input_fn=lambda: input_fn(train))
# .predict() returns an iterator; convert to an array
y_hat = np.array(list(itertools.islice(temp, 0, None)))
train_pred["y_hat"] = y_hat

# Use median value by id
y_hat_med = train_pred.groupby("id").median()["y_hat"].to_dict()

RMSE = np.sqrt(mean_squared_error(train_pred["id"].map(y_hat_med).values, train["y"]))
        
# Prune outliers
RMSE_decreasing = True
count = 0
while (RMSE_decreasing):
    count +=1
    train_pred["y_med"] = train_pred["id"].map(y_hat_med)

    # Distance from the median for each bag
    train_pred["score"] = (train_pred["y_hat"] - train_pred["y_med"])**2
    # Rank of each instance by bag
    train_pred["rank"] = train_pred.groupby("id")["score"].rank()
    bag_size_dict = train_pred.groupby("id")["score"].count().to_dict()
    train_pred["bag_size"] = train_pred["id"].map(bag_size_dict)
    train_pred["rank"] = train_pred["rank"] / train_pred["bag_size"]

    # Remove outliers
    outliers_index = train_pred["rank"] > (1 - best_parameters["outliers_threshold"][0])
    train = train.loc[~outliers_index, :].reset_index(drop=True)

    model_dnn = tf.contrib.learn.DNNRegressor(feature_columns=feature_cols,
                                              hidden_units=[best_parameters["nb_neurons_1"][0]],
                                              model_dir=model_dir)

    model_dnn.fit(input_fn=lambda: input_fn(train), steps=best_parameters["steps"][0])

    # Compute new RMSE
    train_pred = train[["id"]].assign(y_hat=0)
            
    temp = model_dnn.predict(input_fn=lambda: input_fn(train))
    # .predict() returns an iterator; convert to an array
    y_hat = np.array(list(itertools.islice(temp, 0, None)))
    train_pred["y_hat"] = y_hat

    # Use median value by id
    y_hat_med = train_pred.groupby("id").median()["y_hat"].to_dict()

    new_RMSE = np.sqrt(mean_squared_error(train_pred["id"].map(y_hat_med), train["y"]))
    print("Pruning {0} RMSE: {1}".format(count, new_RMSE))

    if (abs(new_RMSE - RMSE) > best_parameters["gain_threshold"][0]):
        RMSE = new_RMSE
    else:
        RMSE_decreasing = False
        
# Bagging of RNN

# Bootstrap 1
#train_1 = train
#model_dnn_1 = tf.contrib.learn.DNNRegressor(feature_columns=feature_cols,
 #                                           hidden_units=[best_parameters["nb_neurons_1"][0]])
#model_dnn_1.fit(input_fn=lambda: input_fn(train_1), steps=best_parameters["steps"][0])
    
models = {"dnn_1": model_dnn}

Pruning 1 RMSE: 0.682372839094
Pruning 2 RMSE: 0.668013398984
Pruning 3 RMSE: 0.659537802041


### Predicting on test set

In [9]:
test_pred = test[["id"]].assign(y_hat=0).reset_index(drop=True)
for i, m in models.items():
    temp = m.predict(input_fn=lambda: input_fn(test))
    # .predict() returns an iterator; convert to an array
    y_hat = np.array(list(itertools.islice(temp, 0, None)))
    test_pred["y_hat"] += models_weights[i] * y_hat

# Use median value by id
y_hat_med = test_pred.groupby("id").median()["y_hat"]

In [10]:
#RMSE = np.sqrt(mean_squared_error(test_pred["id"].map(y_hat_med).values, test["y"]))
#RMSE
#0.65725435012348465 for Pred 4
#0.65362864377856866 for Pred 5

In [11]:
kaggle_pred = pd.DataFrame({"Id": y_hat_med.index, "y": y_hat_med.values})
kaggle_pred.to_csv("Prediction_6.csv", encoding="utf-8", index=False)

### Benchmark:
* Submit 1 (ensemble of xgboost + 2 ridge with instances model)
eta	eval_metric	gamma	lambda	max_depth	min_child_weight	nthread	objective	seed	silent	subsample	score
0.91834 Public LB 300 trees 0.09	rmse	0.2	0.8	4	4.0	-1	reg:linear	22	0	0.7	0.883339 (cross-val)
    
* Submit 2

Pruning with linear regression
then add contributions of aggregated xgboost + linear model

0.78181 Public LB  0.779345 CV

* Submit 3
DNN pruning
(wrong CV)
0.73270 Public LB 0.663128 CV with DNN 10 neurons 900 steps gain_threshold = 0.01 outliers_threshold = 0.05

* Prediction 4
Ensemble of 5 DNN
{'gain_threshold': 0.015, 'steps': 3800, 'nb_neurons_1': 10, 'outliers_threshold': 0.05} CV 0.701658836521
using fist pruning DNN
* Prediction 5
Ensemble of 5 DNN
{'gain_threshold': 0.01, 'steps': 2400, 'nb_neurons_1': 10, 'outliers_threshold': 0.05} CV 0.707300896236